# | HW3 | MobileNetV2 변형해 보기

**Due: 9/26, 11:59 PM**

- **채점 기준**
  - 아래 과제 설명을 따라야한다.
  - test accuracy가 **80% 이상** 나와야 한다.
- **제출**
  - "HW3_학번_이름.ipynb" 형태로 저장하여 Jupyter Notebook을 그대로 제출.
    - 예: HW3_2022_12345_keondo.ipynb
  - output 지우지 말아 주세요

---

`BatchNormalization(axis, momentum, epsilon)` : https://keras.io/api/layers/normalization_layers/batch_normalization/
- axis: Batch normalization이 적용될 axis. 우리는 채널에 대해서 BN을 적용할 것이다. 
- momentum: Moving average에 적용될 momentum 계수
- epsilon: 0으로 나누는 것을 방지하기 위한 작은 수.


`DepthwiseConv2D(kernel_size, strides, padding, use_bias, depthwise_regularizer)` : https://keras.io/api/layers/convolution_layers/depthwise_convolution2d/

paper:[MobileNetV3](https://openaccess.thecvf.com/content_ICCV_2019/papers/Howard_Searching_for_MobileNetV3_ICCV_2019_paper.pdf)  

이번 과제에서는 MobileNetV3에서 추가된 내용 중 일부를 반영해 볼 것이다. MobilenetV3에서는 모델의 마지막 부분에 아래 그림과 같은 변화가 있었는데, 요약하자면
* Average pooling 앞의 1x1 Convolution layer와 Average pooling layer의 순서를 바꾸어 줌으로써 Computation은 줄이면서 정보의 손실은 최소화하였다.
* 위 변화가 일어나게 됨으로써 그 이전 Inverted residual layer에서 projection/filtering을 해 줄 필요가 없어졌다. 따라서 마지막 Inverted residual layer의 Expansion 이후 바로 Average pooling이 오게 된다.
* 아래 그림을 보면 더 이해가 쉬울 것이다.
<img src="https://user-images.githubusercontent.com/37704174/112775642-734f8a80-9078-11eb-9bc1-a860a1fea407.PNG" width="700" height="700"/> 
* 마지막 Inverted residual layer는 Original last stage 그림에서 맨 앞 세개이다.


<br>
위 내용을 참조하여 Network의 마지막 부분을 변형한 MobileNetV2plus를 구성하라. 위 그림상의 H-swish는 고려하지 않아도 된다.
<img src="https://user-images.githubusercontent.com/37704174/112777027-1229b600-907c-11eb-9f89-a7b61c0843be.PNG" width="700" height="700"/>  

- **채점기준**
  - 위의 변경 사항 반영하기
    - MobileNetV2에서 마지막 inverted residual block 및 뒷부분을 고치면 됨
    - Average pooling의 output의 가로 세로는 1임
  - test accuracy **80%** 이상
    - BatchNormalization, Activation, Dropout, Regularization, Weight initialization 등 자유롭게 수정, 추가, 제거 가능
    - `strides` 수정 가능
    - 나머지는 그대로


## Import Modules

In [1]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
### Q1. Import modules ###

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Add, ReLU, Input, Dense, Activation, Flatten, Conv2D, \
    DepthwiseConv2D, BatchNormalization, GlobalAveragePooling2D, Dropout
from tensorflow.keras.regularizers import l2

/Users/liontea/miniconda3/envs/tf2.9.2_p3.10.5/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Inverted Residual Block

- 실습 때 한 것과 동일 

In [2]:
def _inverted_res_block(inputs, expansion, filters, strides):
    x = inputs
    in_chnls = inputs.shape[-1]
    # Expansion
    if expansion != 1:
        x = Conv2D(kernel_size=1, filters=in_chnls * expansion, strides=1, padding='same', use_bias=False, kernel_regularizer=l2(4e-5))(x)
        x = BatchNormalization(momentum=0.999, epsilon=0.001)(x)
        x = ReLU(max_value=6)(x)
        
    # Depthwise convolution
    x = DepthwiseConv2D(kernel_size=3, strides=strides, padding='same', use_bias=False, depthwise_regularizer=l2(4e-5))(x)
    x = BatchNormalization(momentum=0.999, epsilon=0.001)(x)
    x = ReLU(max_value=6)(x)
    
    # Linear bottleneck
    x = Conv2D(kernel_size=1, filters=filters, strides=1, padding='same', use_bias=False, kernel_regularizer=l2(4e-5))(x)
    x = BatchNormalization(momentum=0.999, epsilon=0.001)(x)
    # No activation
    
    # Residual connection
    if in_chnls == filters and strides == 1:
        x = Add()([inputs, x])
        
    return x #return output of layer

In [29]:
def _inverted_res_block_Efficient(inputs, expansion, filters, strides):
    # 마지막 inverted resudial layer의 expansion이후 바로 avergae pooling이 오게 됨
    # -> Depthwise conv와 concat필요 없음
    x = inputs
    in_chnls = inputs.shape[-1]
    
    # Expansion
    if expansion != 1:
        x = Conv2D(kernel_size=1, filters=in_chnls * expansion, strides=1, padding='same', use_bias=False, kernel_regularizer=l2(4e-5))(x)
        x = BatchNormalization(momentum=0.999, epsilon=0.001)(x)
        x = ReLU(max_value=6)(x)
        
    return x #return output of layer

In [30]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
"""
Make sure your runtime type is GPU!
"""
physical_devices = tf.config.list_physical_devices('GPU')
print('Num_GPUs:{}, List:{}'.format(len(physical_devices), physical_devices))
gpu_growth = False

if gpu_growth:
    physical_devices = tf.config.list_physical_devices('GPU')
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except:
        # Invalid device or cannot modify virtual devices once initialized.
        pass

Num_GPUs:1, List:[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## MobileNetV2 변형 구현

In [32]:
def MobileNetV2plus(input_shape, classes):
    inputs = Input(shape=input_shape)

    x = Conv2D(filters=32, kernel_size=3, strides=2, padding='same')(inputs)
    x = BatchNormalization(momentum=0.999, epsilon=0.001)(x)
    x = ReLU(max_value=6)(x)
    
    # inverted residual blocks
    x = _inverted_res_block(inputs = x, expansion=1, filters=16, strides=1)

    x = _inverted_res_block(inputs = x, expansion=6, filters=24, strides=2)
    x = _inverted_res_block(inputs = x, expansion=6, filters=24, strides=1)
    x = Dropout(0.5)

    x = _inverted_res_block(inputs = x, expansion=6, filters=32, strides=2)
    x = _inverted_res_block(inputs = x, expansion=6, filters=32, strides=1)
    x = _inverted_res_block(inputs = x, expansion=6, filters=32, strides=1)
    x = Dropout(0.5)

    x = _inverted_res_block(inputs = x, expansion=6, filters=64, strides=2)
    x = _inverted_res_block(inputs = x, expansion=6, filters=64, strides=1)
    x = Dropout(0.5)
    x = _inverted_res_block(inputs = x, expansion=6, filters=64, strides=1)
    x = _inverted_res_block(inputs = x, expansion=6, filters=64, strides=1)

    x = _inverted_res_block(inputs = x, expansion=6, filters=96, strides=1)
    x = _inverted_res_block(inputs = x, expansion=6, filters=96, strides=1)
    x = Dropout(0.5)
    x = _inverted_res_block(inputs = x, expansion=6, filters=96, strides=1)

    x = _inverted_res_block(inputs = x, expansion=6, filters=160, strides=2)
    x = _inverted_res_block(inputs = x, expansion=6, filters=160, strides=1)
    x = _inverted_res_block_Efficient(inputs = x, expansion=6, filters=160, strides=1)

    ######################################################### - where we modifiy
    
    #1 global average pooling
    x = GlobalAveragePooling2D(keepdims=True)(x)
    
    #2 1x1Conv BN ReLU
    x = Conv2D(kernel_size=1, filters=1280, strides=1, padding='same')(x)
    x = BatchNormalization(momentum=0.999, epsilon=0.001)(x)
    x = ReLU(max_value=6)(x)
    
    #3 FC layer
    outputs = Dense(classes, activation='softmax')(x)
    
    return Model(inputs=inputs, outputs=outputs)

In [34]:
my_mobilenet = MobileNetV2plus((32,32,3),classes=10)
# my_mobilenet.summary()

## Training Data

- keras dataset 혹은 tensorflow dataset 이용
- train data를 9:1로 나눠서 validation data로 이용

In [39]:
from tensorflow.keras.datasets import cifar10
#Load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

#Split train set into train/valid set
from sklearn import model_selection
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(x_train, y_train,test_size=0.1)

## Data Preprocessing
자유롭게 전처리

In [40]:
### Q3. Preporcessing ###
#Data preprocessing/augmentation for training images
import numpy as np
print ("mean before normalization:", np.mean(x_train)) 
print ("std before normalization:", np.std(x_train))

mean=[0,0,0]
std=[0,0,0]
newX_train = np.ones(x_train.shape)
newX_valid = np.ones(x_valid.shape)
newX_test = np.ones(x_test.shape)
#train set에 있는 데이터로만 평균과 표준편차를 구함
for i in range(3):
    mean[i] = np.mean(x_train[:,:,:,i])
    std[i] = np.std(x_train[:,:,:,i])

#train과 test셋 모두 정규화 작업    
for i in range(3):
    newX_train[:,:,:,i] = x_train[:,:,:,i] - mean[i]
    newX_train[:,:,:,i] = newX_train[:,:,:,i] / std[i]
    newX_valid[:,:,:,i] = x_valid[:,:,:,i] - mean[i]
    newX_valid[:,:,:,i] = newX_valid[:,:,:,i] / std[i]
    newX_test[:,:,:,i] = x_test[:,:,:,i] - mean[i]
    newX_test[:,:,:,i] = newX_test[:,:,:,i] / std[i]
        
x_train = newX_train
x_valid = newX_valid
x_test = newX_test

print ("mean after normalization:", np.mean(x_train))
print ("std after normalization:", np.std(x_train))
print(x_train.max())

# ---
# set up image augmentation
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=35,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    )
    
datagen.fit(x_train)
datagen.fit(x_valid)
datagen.fit(x_test)
#########################

mean before normalization: 120.73160056423612
std before normalization: 64.1701416551686
mean after normalization: -3.0270436363261307e-17
std after normalization: 0.999999999999997
2.125997496782931


## Model Compile
loss function, optimizer 설정

In [41]:
# Learning rate decay function
def decay(epoch):
    ####### 실습 #######
    if epoch < 10:
        return 1e-3
    elif epoch < 30:
        return 1e-4
    else:
        return 1e-5
    ###################
    
import math
def exp_decay(epoch):
    ####### 실습 #######
    if epoch < 10:
        return 0.001
    else:
        return 0.001 * math.exp(0.5 * (10-epoch))
    ###################

In [42]:
### Q4. Model compile ###
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
my_mobilenet.compile(loss=loss_fn, optimizer='adam', metrics=['accuracy'])
#########################

In [43]:
### Q5. Callbacks ###
callbacks = [
    tf.keras.callbacks.LearningRateScheduler(exp_decay), 
    tf.keras.callbacks.EarlyStopping(monitor="loss", patience=5),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=7)
]
#####################

## Model Training
hyperparameter를 적절히 설정한다. (epochs 등..)

In [44]:
### Q6. Training ###
history = my_mobilenet.fit(x_train, y_train, batch_size= 64, 
                          epochs= 10,
                          callbacks=callbacks,              
                          validation_data=(x_valid, y_valid))
####################

Epoch 1/10


2022-09-24 12:47:34.201896: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-24 12:47:36.348211: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


704/704 [==============================] - ETA: 0s - loss: 2.0289 - accuracy: 0.1010

2022-09-24 12:48:26.705093: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


704/704 [==============================] - 55s 61ms/step - loss: 2.0289 - accuracy: 0.1010 - val_loss: 2.4837 - val_accuracy: 0.1078 - lr: 0.0010
Epoch 2/10
704/704 [==============================] - 40s 56ms/step - loss: 1.7311 - accuracy: 0.1014 - val_loss: 2.5252 - val_accuracy: 0.1020 - lr: 0.0010
Epoch 3/10
549/704 [======================>.......] - ETA: 8s - loss: 1.5264 - accuracy: 0.1026

## 참고용
조교가 학습한 모델의 validation accuracy를 그래프로 나타내 보았다.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['val_accuracy'])
plt.title('Validation Accuracy', fontsize=15)
plt.xlabel('epochs', fontsize=15)
plt.ylabel('Acc.', fontsize=15)

plt.show()

## Test Accuracy

test accuracy 측정 결과 **80% 이상**이 나와야 한다.

In [ ]:
my_mobilenet.evaluate(x_test,y_test)